In [38]:
#Para hacer scraping de la página del Boletín oficial (búsqueda avanzada)
#Selenium es un conjunto de herramientas de software de código abierto utilizado para la 
#automatización de pruebas funcionales en aplicaciones web. Selenium proporciona una 
#interfaz de programación de aplicaciones (API) para controlar un navegador web y simular 
#la interacción humana con una página web.

#Importar librerías
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

#Lista vacía
dfs = []

# configurar el webdriver
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service)

# abrir la página web
driver.get("https://www.boletinoficial.gob.ar/busquedaAvanzada/primera")

#Definir temas a buscar (en el campo palabra clave). La idea es buscar por tema, sin definir tipo de norma
temas= ['precios'
        , 'multas'
        , 'agio'
        , 'especulación'
        , 'abastecimiento'
#        , 'inflación'
       ]

#Definir años de búsqueda
año_inicio = 1900
año_fin = 1976
años = list(range(año_inicio, año_fin))
años = list(map(str, años))

# buscar elementos 
#itera sobre los temas
for j in temas:
    #Para cada tema, si lo encuentra, intenta iterar sobre los años
    for i in años:
    
        # Cambo de búsqueda palabra clave
        search_box1 = driver.find_element(By.ID, "palabraClave")
        search_box1.send_keys(j)

        # Cambo de búsqueda palabra año
        search_box2 = driver.find_element(By.ID, "anioNormaIP")
        search_box2.send_keys(i)
     
        try:
            # hacer clic en el botón de búsqueda
            search_button = WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.ID, "btnBusquedaAvanzada")))
            search_button.click()
            
            # espera a que se carguen nuevos resultados (1 segundo)
            time.sleep(1)
            
            # limpiar ambos campos de entrada
            search_box1.clear()
            search_box2.clear()
           
            
            # hacer scroll hasta el final de la página
            while True:
                # obtener número actual de resultados
                num_resultados = len(driver.find_elements(By.XPATH, "//div[@class='list-group-item list-group-item-action flex-column align-items-start']"))
    
                # hacer scroll hasta el final de la página
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
                # esperar a que se carguen nuevos resultados (2 segundos)
                time.sleep(2)
    
                # obtener nuevo número de resultados
                num_nuevos_resultados = len(driver.find_elements(By.XPATH, "//div[@class='list-group-item list-group-item-action flex-column align-items-start']"))
    
                # salir del bucle si no hay más resultados nuevos
                if num_nuevos_resultados == num_resultados:
                    break

            # extraer resultados visibles
            resultados = driver.find_elements(By.XPATH, "//div[contains(@class, 'col-md-12') and @class!='row']")

            #Busca el elemento "8" entre los resultados, que es el que contiene toda la normativa
            normas = resultados[8].text
            #Separa la gran string de normas en una lista
            lista_normas = normas.split('\n')

            #Se eliminan de la lista los elementos que indican los tipos de normas que siguen
            for item in ['LEYES', 'DECRETOS', 'RESOLUCIONES', 'DISPOSICIONES', 
                         'RESOLUCIONES GENERALES', 'RESOLUCIONES CONJUNTAS', 'AVISOS OFICIALES']:
                if item in lista_normas:
                    lista_normas.remove(item)
            
            #Se crean 4 columnas con los 4 elementos de cada norma
            original = lista_normas
            n = 4
            sublistas = []

            for i in range(0, len(original), n):
                sublistas.append(original[i:i+n])

            # Crear la lista de listas
            lista_de_listas = sublistas

            # Crear el DataFrame
            df = pd.DataFrame(data=lista_de_listas, columns=['Poder', 'Norma', 'Fecha', 'Contenido'])
            df['tema'] = j

            dfs.append(df)
        
        except:
            # si no se encuentra la palabra clave, pasar a la siguiente iteración sin generar un mensaje de error
            pass

In [ ]:
#crea un dataframe con las normas ya divididas en 4 columnas
dfs_concatenado = pd.concat(dfs)
dfs_concatenado = dfs_concatenado.reset_index()
dfs_concatenado = dfs_concatenado.drop('index', axis=1)

In [41]:
#Procesamiento de lenguaje natural con spacy
#SpaCy es una biblioteca de procesamiento del lenguaje natural (NLP) 
#de código abierto para Python. SpaCy se utiliza para realizar tareas 
#de procesamiento de texto, como tokenización, análisis sintáctico, 
#etiquetado POS (partes del habla), reconocimiento de entidades nombradas, 
#desambiguación de sentidos, entre otras.

#SpaCy también cuenta con modelos pre-entrenados para varios idiomas, 
#incluidos inglés, español, alemán, francés, portugués, italiano y holandés, 
#lo que permite a los usuarios realizar tareas de NLP sin necesidad de entrenar modelos desde cero.

In [40]:
from math import isnan
from unidecode import unidecode
import spacy

In [42]:
# Cargar modelo de lenguaje en español
nlp = spacy.load('es_core_news_sm')

In [43]:
# Definir función para analizar texto
def analizar_texto(texto):
    doc = nlp(texto)
    #usar el modelo para extraer diversas entidades del resumen del contenido de cada norma 
    #Obtener sustantivos y nombres propios, verbos y adjetivos
    sustantivos_y_nombres_propios = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'PROPN']]
    verbos = [token.lemma_ for token in doc if token.pos_ == 'VERB']
    adjetivos = [token.text for token in doc if token.pos_ == 'ADJ']
    # Encontrar caracteres numéricos 
    #(que normalmente representan otras normas a las que hace referencia la que estamos analizando)
    numeros = [token.text for token in doc if token.is_digit]
    return sustantivos_y_nombres_propios, verbos, adjetivos, numeros

#crear 4 columnas nuevas con las distintas entidades aplicando la función a la columna 'contenido' 
dfs_concatenado['Contenido'].fillna('', inplace=True)
dfs_concatenado[['sustantivos_y_nombres_propios', 'verbos', 'adjetivos', 'numeros']] = pd.DataFrame(dfs_concatenado['Contenido'].apply(analizar_texto).tolist())

# Ahora viene mucho trabajo manua, tratando de identificar sustantivos, vervos y adjetivos de interés manualmente. Además de corregir algunas palabras mal categorizadas

In [44]:
# crear una lista de sustantivos para ver (a mano) cuáles de ellos referencian productos y otras entidades de interés
# La columna de sustantivos contine listas de aquellos que aparecen en cada decreto
lista_sustantivos = dfs_concatenado['sustantivos_y_nombres_propios'].explode()

# Convertir la lista en un conjunto para eliminar duplicados
conjunto_sustantivos = set(lista_sustantivos)

# Convertir el conjunto de nuevo en una lista
lista_final = list(conjunto_sustantivos)

# Crear un nuevo DataFrame con la lista de sustantivos únicos
df_sust = pd.DataFrame({'sustantivos': lista_final})

#Este dataframe fue originalmente exportado a excel y categorizado 
#(cada búsqueda nueva que se haga puede tomar como base lo ya categorizado)
#llamar al excel, con los productos y otras entidades ya identificados 
sustantivos_lista = pd.read_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\listado de productos.xlsx')

#aver qúe sustantivos de la "lista_final" no están entre los que ya hemos categorizado
#Para eso primero se convierte la columna del excel en una lista
sustantivos_ya_categorizados = sustantivos_lista['sustantivos'].tolist()
#Luego se crea otra lista con aquellos de "lista_final" que no están entre los ya categorizados
no_estan = [sust for sust in lista_final if sust not in sustantivos_ya_categorizados]
#Por último, se crea un dataframe con la lista de los que no están, agregando columnas 
#con categorías vacías para categorizarlos a mano
df_sust_no_estan = pd.DataFrame({'sustantivos': no_estan, 'PRODUCTO': "", 'agio-espec': "", 'precio': "", 'multa': "", 'otra norma': "", 'max_bas': "", 'acciones': "", 'licitación-concurso-adqui': "", 'abastecimiento': ""})

In [45]:
#sumar los nuevos sustantivos
df_sust_nuevo = pd.concat([sustantivos_lista, df_sust_no_estan])
df_sust_nuevo = df_sust_nuevo.reset_index()
df_sust_nuevo = df_sust_nuevo.drop('index', axis=1)

#exportar a excel para proceder a categorizar
df_sust_nuevo.to_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\listado de productos.xlsx', index=False)

# Acá viene un paso intermedio manual, que es el de identificar productos, referencias a agio y especulación, precios, etc. La mayoría están identificados, pero pueden aparecer más dependiendo de los rangos de fechas y palabras claves buscadas

In [46]:
#llamar al excel de nuevo, con los productos agregados en el paso anterior ya identificados
sustantivos_lista = pd.read_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\listado de productos.xlsx')

#Armar listas con los sustantivos que encajan en las categorías definidas
productos = sustantivos_lista[sustantivos_lista['PRODUCTO'] == 1]
sustantivos_excluidos = productos['sustantivos'].tolist()

precio = sustantivos_lista[sustantivos_lista['precio'] == 1]
sustantivos_excluidos_precio = precio['sustantivos'].tolist()

agio_espec = sustantivos_lista[sustantivos_lista['agio-espec'] == 1]
sustantivos_excluidos_agio = agio_espec['sustantivos'].tolist()

multas = sustantivos_lista[sustantivos_lista['multa'] == 1]
sustantivos_excluidos_multas = multas['sustantivos'].tolist()

normas = sustantivos_lista[sustantivos_lista['otra norma'] == 1]
sustantivos_excluidos_normas = normas['sustantivos'].tolist()

licit_concur_adqui = sustantivos_lista[sustantivos_lista['licitación-concurso-adqui'] == 1]
sustantivos_excluidos_licit = licit_concur_adqui['sustantivos'].tolist()

abastecimiento = sustantivos_lista[sustantivos_lista['abastecimiento'] == 1]
sustantivos_excluidos_abast = abastecimiento['sustantivos'].tolist()

#adjetivos en sustantivos
max_bas = sustantivos_lista[sustantivos_lista['max_bas'] == 1]
adj_excluidos1 = max_bas['sustantivos'].tolist()

#verbos en sustantivos
acción = sustantivos_lista[sustantivos_lista['acciones'] == 1]
verb_excluidos1 = acción['sustantivos'].tolist()

todos_los_excluidos = sustantivos_excluidos+sustantivos_excluidos_precio+sustantivos_excluidos_agio+sustantivos_excluidos_multas+sustantivos_excluidos_normas+adj_excluidos1+verb_excluidos1

In [47]:
#Dividir la columna sustantivos en cada categoría y en otros sustantivos
# Crear una lista de sustantivos que están en la lista de excluidos
sustantivos_producto = list(set([sustantivo for sustantivos in dfs_concatenado['sustantivos_y_nombres_propios'] for sustantivo in sustantivos if sustantivo in sustantivos_excluidos]))
sustantivos_precio = list(set([sustantivo for sustantivos in dfs_concatenado['sustantivos_y_nombres_propios'] for sustantivo in sustantivos if sustantivo in sustantivos_excluidos_precio]))
sustantivos_agio = list(set([sustantivo for sustantivos in dfs_concatenado['sustantivos_y_nombres_propios'] for sustantivo in sustantivos if sustantivo in sustantivos_excluidos_agio]))
sustantivos_multa = list(set([sustantivo for sustantivos in dfs_concatenado['sustantivos_y_nombres_propios'] for sustantivo in sustantivos if sustantivo in sustantivos_excluidos_multas]))
sustantivos_normas = list(set([sustantivo for sustantivos in dfs_concatenado['sustantivos_y_nombres_propios'] for sustantivo in sustantivos if sustantivo in sustantivos_excluidos_normas]))
sustantivos_max_bas = list(set([sustantivo for sustantivos in dfs_concatenado['sustantivos_y_nombres_propios'] for sustantivo in sustantivos if sustantivo in adj_excluidos1]))
sustantivos_accion = list(set([sustantivo for sustantivos in dfs_concatenado['sustantivos_y_nombres_propios'] for sustantivo in sustantivos if sustantivo in verb_excluidos1]))
sustantivos_licit = list(set([sustantivo for sustantivos in dfs_concatenado['sustantivos_y_nombres_propios'] for sustantivo in sustantivos if sustantivo in sustantivos_excluidos_licit]))
sustantivos_abast = list(set([sustantivo for sustantivos in dfs_concatenado['sustantivos_y_nombres_propios'] for sustantivo in sustantivos if sustantivo in sustantivos_excluidos_abast]))

todos_los_excluidos = sustantivos_excluidos+sustantivos_excluidos_precio+sustantivos_excluidos_agio+sustantivos_excluidos_multas+sustantivos_excluidos_normas+adj_excluidos1+verb_excluidos1+sustantivos_excluidos_licit+sustantivos_excluidos_abast

# crear una lista de sustantivos que no están en la lista de excluidos
sustantivos_no_ProdPreAgioMult = list(set([sustantivo for sustantivos in dfs_concatenado['sustantivos_y_nombres_propios'] for sustantivo in sustantivos if sustantivo not in todos_los_excluidos]))

# crear nuevas columnas con las listas de sustantivos específicos y los otros
dfs_concatenado['productos'] = dfs_concatenado['sustantivos_y_nombres_propios'].apply(lambda sustantivos: list(set(sustantivos) & set(sustantivos_excluidos)))
dfs_concatenado['precio'] = dfs_concatenado['sustantivos_y_nombres_propios'].apply(lambda sustantivos: list(set(sustantivos) & set(sustantivos_excluidos_precio)))
dfs_concatenado['agio'] = dfs_concatenado['sustantivos_y_nombres_propios'].apply(lambda sustantivos: list(set(sustantivos) & set(sustantivos_excluidos_agio)))
dfs_concatenado['multa'] = dfs_concatenado['sustantivos_y_nombres_propios'].apply(lambda sustantivos: list(set(sustantivos) & set(sustantivos_excluidos_multas)))
dfs_concatenado['norma'] = dfs_concatenado['sustantivos_y_nombres_propios'].apply(lambda sustantivos: list(set(sustantivos) & set(sustantivos_excluidos_normas)))
dfs_concatenado['licit_concur_adqui 1'] = dfs_concatenado['sustantivos_y_nombres_propios'].apply(lambda sustantivos: list(set(sustantivos) & set(sustantivos_excluidos_licit)))
dfs_concatenado['abastecimiento'] = dfs_concatenado['sustantivos_y_nombres_propios'].apply(lambda sustantivos: list(set(sustantivos) & set(sustantivos_excluidos_abast)))
dfs_concatenado['adj max bas 1'] = dfs_concatenado['sustantivos_y_nombres_propios'].apply(lambda sustantivos: list(set(sustantivos) & set(adj_excluidos1)))
dfs_concatenado['acciones 1'] = dfs_concatenado['sustantivos_y_nombres_propios'].apply(lambda sustantivos: list(set(sustantivos) & set(verb_excluidos1)))
dfs_concatenado['otros sustantivos'] = dfs_concatenado['sustantivos_y_nombres_propios'].apply(lambda sustantivos: list(set(sustantivos) - set(todos_los_excluidos)))


# VERBOS

In [48]:
#hacemos algo similar con los verbos
lista_verbos = dfs_concatenado['verbos'].explode()

conjunto_verbos = set(lista_verbos)


lista_final = list(conjunto_verbos)


df_verbos = pd.DataFrame({'verbos': lista_final})


df_verbos.to_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\verbos_unicos.xlsx', index=False)


acciones_lista = pd.read_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\listado de acciones.xlsx')


verbos_ya_categorizados = acciones_lista['verbos'].tolist()

no_estan = [verb for verb in lista_final if verb not in verbos_ya_categorizados]

df_verb_no_estan = pd.DataFrame({'verbos': no_estan, 'ACCION': "",  'max-bas': "", 'licitación-concurso-adqui': ""})

In [49]:

df_verb_nuevo = pd.concat([acciones_lista, df_verb_no_estan])
df_verb_nuevo = df_verb_nuevo.reset_index()
df_verb_nuevo = df_verb_nuevo.drop('index', axis=1)


df_verb_nuevo.to_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\listado de acciones.xlsx', index=False)

# Acá viene un paso intermedio manual, que es el de identificar acciones de interés. La mayoría están identificados, pero pueden aparecer más dependiendo de los rangos de fechas y palabras claves buscadas

In [50]:

acciones_lista = pd.read_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\listado de acciones.xlsx')

acciones = acciones_lista[acciones_lista['ACCION'] == 1]
verbos_excluidos = acciones['verbos'].tolist()

licit_concur_adqui = acciones_lista[acciones_lista['licitación-concurso-adqui'] == 1]
verb_excluidos_licit = licit_concur_adqui['verbos'].tolist()


max_bas = acciones_lista[acciones_lista['max-bas'] == 1]
adj_excluidos2 = max_bas['verbos'].tolist()

In [51]:

verbos_acción = list(set([verbo for verbo in dfs_concatenado['verbos'] for verbo in verbo if verbo in verbos_excluidos]))
adj_lista_verb = list(set([adj for adj in dfs_concatenado['verbos'] for adj in adj if adj in adj_excluidos1]))
licit_cocnur_adqui = list(set([verbo for verbo in dfs_concatenado['verbos'] for verbo in verbo if verbo in verb_excluidos_licit]))

todos_los_verb_excluidos = verbos_excluidos+adj_excluidos1+verb_excluidos_licit


verbos_no_acción = list(set([verbo for verbo in dfs_concatenado['verbos'] for verbo in verbo if verbo not in verbos_excluidos]))


dfs_concatenado['acciones 2'] = dfs_concatenado['verbos'].apply(lambda verbos: list(set(verbos) & set(verbos_excluidos)))
dfs_concatenado['licit_concur_adqui 2'] = dfs_concatenado['verbos'].apply(lambda verbos: list(set(verbos) & set(verb_excluidos_licit)))
dfs_concatenado['adj max bas 2'] = dfs_concatenado['verbos'].apply(lambda verbos: list(set(verbos) & set(adj_excluidos2)))
dfs_concatenado['otros verbos'] = dfs_concatenado['verbos'].apply(lambda verbos: list(set(verbos) - set(todos_los_verb_excluidos)))


In [52]:
def unir_listas(row, cols):
    lista_unida = []
    for col in cols:
        lista_unida.extend(row[col])
    return lista_unida

In [53]:
#Finalmente podemos crear una columna que contenga referencias a licitaciones
dfs_concatenado['licit_concur_adqui'] = dfs_concatenado.apply(unir_listas, axis=1, args=(['licit_concur_adqui 1', 'licit_concur_adqui 2'],))

In [54]:
#Finalmente podemos crear una columna que contenga las acciones
dfs_concatenado['acciones'] = dfs_concatenado.apply(unir_listas, axis=1, args=(['acciones 1', 'acciones 2'],))

# adjetivos

In [55]:

lista_adj = dfs_concatenado['adjetivos'].explode()


conjunto_adj = set(lista_adj)


lista_final = list(conjunto_adj)


df_adj = pd.DataFrame({'adjetivos': lista_final})


df_adj.to_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\adjetivos_unicos.xlsx', index=False)


adj_lista = pd.read_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\listados max_bas.xlsx')


adj_ya_categorizados = adj_lista['adjetivos'].tolist()

no_estan = [adj for adj in lista_final if adj not in adj_ya_categorizados]

df_adj_no_estan = pd.DataFrame({'adjetivos': no_estan, 'max_bas': ""})

In [56]:

df_adj_nuevo = pd.concat([adj_lista, df_adj_no_estan])
df_adj_nuevo = df_adj_nuevo.reset_index()
df_adj_nuevo = df_adj_nuevo.drop('index', axis=1)


df_adj_nuevo.to_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\listados max_bas.xlsx', index=False)

# Acá viene un paso intermedio manual, que es el de identificar maximos, minimos, básicos, etc. La mayoría están identificados, pero pueden aparecer más dependiendo de los rangos de fechas y palabras claves buscadas

In [57]:

adj_lista = pd.read_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\listados max_bas.xlsx')
max_bas = adj_lista[adj_lista['max_bas'] == 1]
adj_excluidos3 = max_bas['adjetivos'].tolist()

In [58]:
adj_lista = list(set([adj for adj in dfs_concatenado['adjetivos'] for adj in adj if adj in adj_excluidos3]))

# Creamos una lista de verbos que no están en la lista de excluidos
adj_no_lista = list(set([adj for adj in dfs_concatenado['adjetivos'] for adj in adj if adj not in adj_excluidos3]))

# Creamos dos nuevas columnas con las listas de verbos
dfs_concatenado['adj max bas 3'] = dfs_concatenado['adjetivos'].apply(lambda verbos: list(set(verbos) & set(adj_excluidos3)))
dfs_concatenado['otros adjetivos'] = dfs_concatenado['adjetivos'].apply(lambda verbos: list(set(verbos) - set(adj_excluidos3)))


In [59]:
#Finalmente podemos crear una columna que contenga el tipo de adjetivos de interés
dfs_concatenado['adj max bas'] = dfs_concatenado.apply(unir_listas, axis=1, args=(['adj max bas 1', 'adj max bas 2', 'adj max bas 3'],))

In [60]:
#mejorar algunas columnas
dfs_concatenado[['Norma', 'Año']] = dfs_concatenado['Norma'].str.split('/', n=1, expand=True)
dfs_concatenado[['Norma', 'Número']] = dfs_concatenado['Norma'].str.split(' ', n=1, expand=True)
dfs_concatenado['Fecha'] = dfs_concatenado['Fecha'].str.replace('Fecha de Publicacion: ', '')

In [61]:
#Paso manual para corregir la escritura d elas acciones de interés
acciones_originales = dfs_concatenado['acciones'].explode()
acciones_originales = [valor for valor in acciones_originales if valor == valor]

acciones_originales = sorted(list(set(acciones_originales)))

In [62]:
print(acciones_originales)

['-modificación', '11,747.fijanse', 'AJUSTAN', 'AJUSTE', 'AUTORIZACION', 'AUTORIZAR', 'AUTORIZASE', 'Ajuste', 'Autoriz', 'Autoriza', 'Autorización', 'Autorizando', 'Autorizanse', 'Autorízase', 'Contabilidad.-Reajuste', 'DEROGASE', 'Deroga', 'Derogación', 'Deroganse', 'Deróganse', 'Derógase', 'Déjanse', 'Déjase', 'EFECTO', 'ESTABLECEN', 'ESTABLECENSE', 'ESTABLECER', 'ESTABLECIDO', 'ESTABLECIDOS', 'Efecto', 'Establecese', 'FIJA', 'FIJADAS', 'FIJADOS', 'FIJAN', 'FIJANSE', 'FIJARAN', 'FIJASE', 'Fija', 'Fijación', 'Fijan', 'Fijando', 'Fijanse', 'Fijase', 'Fíajanse', 'Fíja', 'Fíjanse', 'Fíjase', 'MODIFICACION', 'MODIFICAN', 'MODIFICANSE', 'MODIFICAR', 'Modif', 'Modific', 'Modifica', 'Modificacion', 'Modificación', 'Modificanse', 'Modificase', 'Modifícanse', 'Modifícase', 'REAJUSTASE', 'REAJUSTE', 'Reajusta', 'Reajustanse', 'Reajuste', 'Reajustes', 'Reajústanse', 'Rebaja', 'Rebajas', 'actualizanse', 'actualizar', 'aestableceir', 'ajustar', 'ajustar él', 'ajuste', 'amplianse', 'ampliar', 'auto

In [67]:
acciones_corregidas = ['modificar', 'fijar', 'ajustar', 'ajustar', 'autorizar', 'autorizar', 'autorizar', 
                       'ajustar', 'autorizar', 'autorizar', 'autorizar', 'autorizar', 'autorizar', 'autorizar', 
                       'reajustar', 'derogar', 'derogar', 'derogar', 'derogar', 'derogar', 'derogar', 
                       'dejar', 'dejar', 'efecto', 'establecer', 'establecer', 'establecer', 'establecer', 
                       'establecer', 'efecto', 'establecer', 'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 
                       'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 
                       'fijar', 'modificar', 'modificar', 'modificar', 'modificar', 'modificar', 'modificar', 'modificar', 
                       'modificar', 'modificar', 'modificar', 'modificar', 'modificar', 'modificar', 'reajustar', 
                       'reajustar', 'reajustar', 'reajustar', 'reajustar', 'reajustar', 'reajustar', 'rebajar', 'rebajar', 
                       'actualizar', 'actualizar', 'establecer', 'ajustar', 'ajustar', 'ajustar', 'ampliar', 
                       'ampliar', 'autorizar', 'autorizar', 'autorizar', 'dejar', 'dejar', 'derogar', 'derogar', 
                       'derogar', 'derogar', 'derogar', 'determinar', 'determinar', 'efecto', 'eliminar', 
                       'eliminar', 'elevar', 'establecer', 'establecer', 'establecer', 'establecer', 'establecer', 
                       'establecer', 'establecer', 'establecer', 'excluir', 'excluir', 'excluir', 'excluir', 'fijar', 
                       'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 'fijar', 'flexibilizar', 
                       'fijar', 'fijar', 'incluir', 'incluir', 'incluir', 'incluir', 'incorporar', 'incrementar', 
                       'liberar', 'liberar', 'liberar', 'modificar', 'modificar', 'modificar', 'modificar', 
                       'modificar', 'modificar', 'modificar', 'modificar', 'modificar', 'modificar', 'reajustar', 
                       'reajustar', 'rebajar', 'rebajar', 'suprimir']

In [ ]:
acciones_corregidas = pd.DataFrame({'acciones de interés': acciones_originales, 'acciones_corregidas': acciones_corregidas})

In [72]:
dfs_concatenado['acciones corregidas'] = dfs_concatenado['acciones']

# Crea el diccionario de reemplazo
reemplazo = dict(zip(acciones_corregidas['acciones de interés'], acciones_corregidas['acciones_corregidas']))

# Aplica la función de reemplazo en cada celda del DataFrame
dfs_concatenado['acciones corregidas'] = dfs_concatenado['acciones corregidas'].apply(lambda x: [reemplazo.get(i, i) for i in x])

In [73]:
dfs_concatenado["Año"] = dfs_concatenado["Año"].str.replace("N/", "")
dfs_concatenado["Año"] = dfs_concatenado["Año"].str.replace("n/", "")

dfs_concatenado['Año'] = pd.to_numeric(dfs_concatenado['Año'] , errors='coerce').astype('Int64')

In [74]:
#Guardamos nuestro dataset con información categorizada de decretos de diversos temas y 
#con la amplitud de fechas que queramos

dfs_concatenado.to_excel(r'C:\Users\lucat\OneDrive\HEAnuevo\decretos 1900-1975.xlsx', index=False)



## FIN